# Import / Params

In [ ]:
import polars as pl
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open('../params.json', 'r') as file :
    params = json.load(file)

DATASET, VERSION = params['dataset'], params['version']
DATA_FOLD = params['data_folder']

In [ ]:
static = pl.read_parquet(f'{DATA_FOLD}/{VERSION}/2.clean_data/{DATASET}/static/clean_static_encounters.parquet')
temporal = pl.read_parquet(f'{DATA_FOLD}/{VERSION}/2.clean_data/{DATASET}/temporal/all_features_with_delta.parquet')

# Pré-traitement

## Filtrage des patients 'valides'

In [ ]:
# On considère comme valide les patients avec au moins une valeur de SpO2 ou de Fréquence cardiaque

# Liste des patients correspondant à ce critère
valid_patients = temporal.filter(
    (pl.col("heart_rate").is_not_null()) | (pl.col("spo2").is_not_null())
).select("encounterId").unique()

# Filtre du dataset à partir de la liste
temporal_valid = temporal.filter(pl.col('encounterId').is_in(valid_patients.to_series().to_list()))

## Définition du premier/dernier timestamp valide

In [ ]:
# On définit le premier timestamp (ts) valide comme le premier ayant une valeur de SpO2 ou une fréquence cardiaque

# On identifie les timestamps de chaque patient correspondant à ce critère

valid_ts = temporal.with_columns(
    pl.when(
        (pl.col("heart_rate").is_not_null()) | (pl.col("spo2").is_not_null()) | (pl.col("pam_invasive").is_not_null()) | (pl.col("pam_non_invasive").is_not_null())
        )
    .then(pl.col("delta_hour"))
    .otherwise(None)
    .alias("valid_timestamp")
)

# On ne garde que le plus petit pour chaque encounterId dans un dataframe séparé
first_ts = valid_ts.group_by("encounterId").agg(
    [
        pl.col("valid_timestamp").min().alias("first_timestamp"),
        pl.col("valid_timestamp").max().alias("last_timestamp"),
    ]
)

# Jointure du dataframe comprenant les plus petits ts valides. Création de l'offset sur delta hour
offset_delta = (temporal_valid
    .join(first_ts, on="encounterId", how="left")
    .with_columns(
            (pl.col("delta_hour") - pl.col("first_timestamp")).alias("delta_hour"),
            (pl.col("last_timestamp") - pl.col("first_timestamp")).alias("last_timestamp")
        )
    .drop("first_timestamp")
)



In [ ]:
max_valid_hours = (
    offset_delta.group_by('encounterId').agg(pl.col('last_timestamp').max())
)

range_rows = [
    {"encounterId": row[0], "delta_hour": h}
    for row in max_valid_hours.iter_rows()
    for h in range(row[1] + 1)
]

neg_delta = offset_delta.filter(pl.col("delta_hour") < 0)

ranges = pl.DataFrame(range_rows)

filled_with_missing_ts = ranges.join(offset_delta, on=["encounterId", "delta_hour"], how="left").cast({'delta_hour': pl.Int32})

df_all_ts = pl.concat([neg_delta, filled_with_missing_ts], how="vertical_relaxed").sort(["encounterId", "delta_hour"])

In [ ]:
merge_pressure = df_all_ts.with_columns(
    pl.when(pl.col('pas_invasive').is_not_null()).then(pl.col('pas_invasive')).when(pl.col('pas_non_invasive').is_not_null()).then(pl.col('pas_non_invasive')).otherwise(None).alias('pas'),
    pl.when(pl.col('pad_invasive').is_not_null()).then(pl.col('pad_invasive')).when(pl.col('pad_non_invasive').is_not_null()).then(pl.col('pad_non_invasive')).otherwise(None).alias('pad'),
    pl.when(pl.col('pam_invasive').is_not_null()).then(pl.col('pam_invasive')).when(pl.col('pam_non_invasive').is_not_null()).then(pl.col('pam_non_invasive')).otherwise(None).alias('pam'),
).drop(
    'pas_invasive', 'pas_non_invasive', 'pad_invasive', 'pad_non_invasive', 'pam_invasive', 'pam_non_invasive'
)

In [ ]:
merge_eer = merge_pressure.with_columns(
    pl.when(pl.col('db_sang_cvvhf').is_not_null()).then(pl.lit('cvvhf'))
    .when(pl.col('db_sang_hdi').is_not_null()).then(pl.lit('hdi'))
    .otherwise(pl.lit(None)).alias('eer')
).drop(
    'db_sang_cvvhf', 'db_sang_hdi'
)

In [ ]:
prone = merge_eer.with_columns(
    pl.when(pl.col('installation').is_not_null()).then(True).otherwise(False).alias('is_prone')
).drop('installation')

In [ ]:
# Modes contrôlés
controlled_modes = [
    'Ventilation Contrôlée', 'Pression Contrôlée', 'PC', 'VC', 'PC Mode pédiatrique',
    'PC VG', 'Pression Contrôlée Volume Garanti', 'Ventilation Oscillatoire à Haute Fréquence',
    'Ventilation Oscillatoire à Haute Fréquence Volume Garanti', 'Jet ventilation à haute fréquence',
    'APRV', 'PC APRV', 'BI-VENT/ APRV', 'BIVENT/APRV', 'PC - APRV', 'PC-BIPAP', 'bipap pc', 'VPC',
    'Bivent', "Ventilation d'apnée", 'S/T', 'VNI PC', 'AVAPS',
    'Ventilation Assistée Contrôlée (VAC)', 'VAC', 'Ventilation Assistée Contrôlée Intermittente (VACI)',
    'VACI', 'vac sur fibroaspiration'
]

# Modes assistés
assisted_modes = [
    'Ventilation Spontanée Aide Inspiratoire (VSAI)', 'VSAI avec hoquet', 'VSAI pro',
    'AI pro', 'AI-pro', 'Ai Pro', 'AI PRO', 'VAI pro', 'aipro', 'AIPro',
    'NAVA', 'Ventilation Non Invasive NAVA', 'SPN-VS-PEP/AI', 'SPN-VS-PEP',
    'VS-PEP / Optiflow', 'VS CP', 'AI/VS PEP'
]

# OHD
ohd_modes = [
    'OHD', 'Ventilation spontanée + pression expiratoire positive / OHD',
    'OHD + ballonnet dégonflé','Optitrach', 'optitrach', 'Opti trach', 'Opti-trach', 'Trach-flow',
]

# VNI/CPAP
vni_cpap_modes = [
    'VNI', 'Ventilation Non Invasive', 'VNI perso', 'VNI SAS perso', 'Ventilation Non Invasive Pression controlée',
    'BiPAP', 'bipap', 'BIPAP', 'bilevel', 'bilevel/aprv', 'CPAP', 'cpap', 'Cpap', 'C-pap',
    'C PAP', 'c PAP', 'PPC', 'CP', 'CPAP perso', 'cpap perso', 'cpap Perso', 'CPAP Perso',
    'CPAP personnelle', 'CPAP personelle', 'CPAP PERSO', 'CiPAP', 'CIPAP', 'cipap', 'CiPaP',
    'CiPap', 'Ci-PAP', 'ci pap', 'Cipap', 'CiPap perso', 'Cipap perso', 'CiPAP perso', 'cipap perso',
    'Cipap Perso', 'CIPAP perso', 'CIPAP Perso', 'CIPAP PERSO', 'CIPAP perso 1h', 'CIPAP personnelle',
    'CiPAP perso', 'CIPPAP', 'CiPaP perso', 'ci pap', 'ciPap', 'CI-PAP', 'cIPAP', 'Valve Boussignac',
    'VNI Helmet', 'V60', 'PPC domicile', 'cpap/VS', 'Cpap Perso', 'VAC s/ VNI', 'NNI 50% 8AI 8PEP'
]

# Modes sevrage
weaning_modes = [
    'Tube en T', 'tube en t', 'TUBE EN T', 'tube en T', 'Tube T', 'T-TUBE', 'tube T O2 5L/min', 'tube en T 5l/min',
    'embout t', 'Embout en T', 'Enbout en T', 'test 7/0', 'TEST 7/0', 'Test 7/0', 'test 7-0', 'test 8-0',
    'test ZEP', 'tEST DE ZEEP', 'Test Zeep', 'test zeep', 'TEST DE ZEEP', 'test de capnie', 'Test tube en T', 'test de sevrage tube T', 'test sevrage',
    'Test de sevrage ventilatoire', 'épreuve de sevrage', 'Test', 'TEST SEVRAGE', 'stop VSAI',
    'Test de sevrage avec filtre', 'séance de déventilation', 'Déventilation', 'EOT par MDG',
    'Echec VSAI', '+ test hypercapnie', 'Test de ZEEP'
]

# Non assistés par un dispositif
non_assisted_modes = [
    'None', 'Aucun/arrêt ventilateur', 'Arrêt ventilateur', 'auto EOT', 'EOT', 'extubation', 'extubé',
    'Extubation 14h', 'Extubation terminale', 'auto-extubation', 'auto extubation', 'décannulation',
    'décanuler', 'Décanulation', 'air ambiant', 'airpo', 'lunettes', 'lunettes a 4 l', 'lunettes a 6 litres',
    'lunette 4l', 'lunettes 4L/min', 'Lunettes', 'nez', 'Pour repas', 'Trachvent', 'trachvent', 'trach vent',
    'trach vent 1L', 'trach vent 2L', 'trach vent 4L', 'Trach Vent', 'Trach Vent 4L', 'trach vent 4l O2',
     'Canule phonatoire',
    'VS canule phonatoire', 'Canule pho obstruée', 'canule pho obstruée', 'Canule phonoatoire obturée',
    'CP Obstruée', 'ballonnet dégonflé', 'Ballonet dégonflé', 'ballonnet gonflée', 'Ballonet gonflé',
    'O2 4L tracheo', '3l o2 embout T', 'MMC', 'MMC 3,5L', 'KINE R', 'chryco BAVU', 'BAVU', 'trackvent',
    'track vent', 'tach vent', 'tachvent', 'Aérosol Nacl', 'VS', 'vs', 'VST', 'Respi perso',
    'intubation en urgence', 'extube', 'Lunettes A 2LITRES', 'LUNETTES A 2LITRES', 'Test hypercapnie', 'test hypercapnie', 
    'Test HyperCO²', 'test hypercapnie'
]


In [ ]:
mode_vent = prone.with_columns(
    pl.when(pl.col('mode_vent').is_in(controlled_modes)).then(pl.lit('Controlled'))
    .when(pl.col('mode_vent').is_in(assisted_modes)).then(pl.lit('Assisted'))
    .when(pl.col('mode_vent').is_in(ohd_modes)).then(pl.lit('OHD'))
    .when(pl.col('mode_vent').is_in(vni_cpap_modes)).then(pl.lit('VNI/CPAP'))
    .when(pl.col('mode_vent').is_in(weaning_modes)).then(pl.lit('Weaning'))
    .when(pl.col('mode_vent').is_in(non_assisted_modes)).then(pl.lit('Non-assisted'))
    .otherwise(None).alias("mode_vent")
)

In [ ]:
mode_vent

In [ ]:
# Conscient / Normal
conscient_normal = [
    'Conscient', 'conscient', 'Coopérant', 'Dort', 'dort', 'DORT', 'Endormi', 'Somnolent',
    'somnolent', 'Somnolent mais éveillable', 'Dort, réveillable', 'dort, réveillable', 'Dors', 'vigile',
     'éveillée', 'Eveil','Conscient ROS des yeux',
    'Consciente avec sédations'
]

confusion_delire = [
    'Confus', 'confus', 'Confuse', 'confuse', 'Désorienté', 'désorienté', 'desorientation temporelle',
    'Délirante'
]

# Altération légère (confusion, désorientation)
alteration_legere = [
    
    'apathique', 'Asthénique +++', 'Ralentie ++', 
]

# Coma (non sédaté)
tb_vigilance_non_sedate = [
    'Coma', 'coma', 'Coma réactif', 'Coma vigile', 'coma réactif neuro sédaté', 'coma faiblement réactif',
    'Coma aréactif', 'coma aréactif', 'Aréactif', 'Coma hors sédation depuis 8h45',
    'COMA hors sédations 16/08 12h', 'ROS yeux fermés', 'Absence de ROS', "absence d'ouverture des yeux",
    "Pas d'O des Y", 'Aucune ouverture des Y', "pas d'ouverture des yeux", 'Clignement à la menace',
    'Auditif', 'Auditive', 'Réponses aux ordres', 'Vigile', 
    'ouverture yeux spontanée', "Ouverture yeux à l'appel", 'Ouverture des yeux', 'Pas de contact visuel',
    'inconscient', 'insconscient', 'Clonies', 'clonies','crise convulsive', 'post Crise tonico clonique',
    'Arrêt sédations', 'Arrêt des sédations', 'arret des sédations', 'arret ds sédations',
    'arret sédations', 'Arret sédations', 'Arret des sedations', 'STOP sedation', 'stop sedation',
    'stop sédation', 'Stop Sédations', 'Stop sedation', 'Stop sédations', 'ouvre les yeux',
    'Légère sédation', 'Légèrment sédaté', 'Sédation légère, reveillable', 'Sédation séquence rapide',
    'perte de contact' ]

# Coma sous sédation
coma_sous_sedation = [
    'Coma neuro sédaté', 'Coma neuro-sédaté', 'coma neuro sédaté curarisé', 'Coma neurosédaté +/- conscient',
    'neurosédaté', 'Coma analgo sédaté', 'Coma analgo-sédaté', 'Coma analgosédaté', 'Coma analgosédation',
    'Coma analgo sedaté', 'coma analgo sédaté', 'coma analgosédaté', 'coma analgo-sédaté', 'Coma Analgosédaté',
    'coma analago sedaté', 'Coma anlago sédaté', 'Coma analgo sésaté', 'Analgo-Sédation', 'Analgo-sédation',
    'Analgo sédation', 'analgo sédation', 'Analgo sédaté', 'analgo-sédaté', 'analgosédation', 'Analgosédation',
    'Analgosédation*', 'Analgosedation', 'Analgo-sedation', 'Analgo -Sedation', 'analgosedation',
    'ANALGOSEDATION', 'analgo sedation', 'Analgo sédatée', 'Analgo sédaté', 'Légère analgosédation',
    'Neuroanalgesie', 'Analgo sedation', 'analgésie', 'ANALGESIE', 'Analgésie', 'coma + analgésie',
    'Curarisé', 'curarisé', 'Sédations de confort', 'coma sédation de confort', 'sédation de confort',
    'sous propo pour colo', 'Sédation pour test de fuite', 'Sedation paliative', 'Sédation terminale',
    'sédations terminale'
]


# Mort encéphalique / Décès
mort_encephalique_deces = [
    'Mort encéphalique', 'mort encéphalique', 'Mort cérébrale', 'Mort encephalique', 'mort encéphlique',
    'MORT ENCEPHALIQUE', 'mORT ENCEPHALIQUE CLINIQUE', 'Mort encéphalique clinique', 'mort encéphalique clinique',
    'Etat de mort encéphalique', 'Etat de mort encéphal clinique', 'état mort encéphalique cliniqu',
    'mort encéphalique paraclinique', 'Décédé', 'DECES', 'DCD le 27.02 à 0h29', 'paetiente décédée',
    'décès', 'ME', 'Diagnostic clinique EME', 'diagnostique clinique ME', 'EME', 'EME clinique',
    'EME Clinique', 'EMC', 'Accompagnement fin de vie', 'Accompgnement fin de vie', 'accompagnement fin de vie',
    'Fin de vie', 'ACR', "Test d'hypercapnie +", 'Test hypercapnie + à 11h', 'Test hypercapnie poitif'
]

# Agitation / Anxiété
agitation_anxiete = [
    'Agité', 'agité', 'reveille agité sans reponses', 'anxieux', 'anxieu', 'anxieuse', 'Anxieuse ++',
    'Anxieux +++', 'angoissée', 'Nerveux', 'nerveux', 'opposant', 'non coopérent aux soins',
    'inconforatable',  'Clonies de la face'
]



In [ ]:
neuro_status = mode_vent.with_columns(
    pl.when(pl.col('neuro_status').is_in(conscient_normal)).then(pl.lit('Conscient/Normal'))
    .when(pl.col('neuro_status').is_in(coma_sous_sedation)).then(pl.lit('Coma sous sédation'))
    .when(pl.col('neuro_status').is_in(alteration_legere)).then(pl.lit('Alteration légère'))
    .when(pl.col('neuro_status').is_in(confusion_delire)).then(pl.lit('Confusion/Delirium'))
    .when(pl.col('neuro_status').is_in(agitation_anxiete)).then(pl.lit('Agitation/Anxiété'))
    .when(pl.col('neuro_status').is_in(tb_vigilance_non_sedate)).then(pl.lit('Altération vigilance (non sédaté)'))
    .when(pl.col('neuro_status').is_in(mort_encephalique_deces)).then(pl.lit('Mort encéphalique / Décès'))
    .otherwise(None).alias("neuro_status")
)

In [ ]:
neuro_status.columns

In [ ]:
neuro_status.filter(pl.col('admin_o2') == 'Air ambiant')

In [ ]:
fio2_corr = neuro_status.with_columns(
    pl.when(pl.col('fio2').is_not_null()).then(pl.col('fio2'))
    .when(pl.col('admin_o2') == 'Air ambiant').then(pl.lit(21))
    .when(pl.col('o2_flow').is_between(0,15)).then((pl.col('o2_flow') * 4 + 21))
    .alias('fio2_corr')
)

In [ ]:
poids_static = static.select(['encounterId', 'poids_admission']).cast(
    {"encounterId" : pl.Int32}
)

poids_corr = fio2_corr.join(poids_static, on='encounterId', how='left').with_columns(
    pl.when((pl.col('poids_suivi').is_null()) & (pl.col('delta_hour') == 0)).then(pl.col('poids_admission'))
    .otherwise(pl.col('poids_suivi')).alias('poids_suivi')
).drop('poids_admission')

In [ ]:
linear_interpolate_both = ['poids_suivi', 'pplat', 'spo2', 'temp', 
        'fr', 'heart_rate' , 'glyc_cap', 'tp', 'creat', 'bili_tot', 'num_plq',
       'leucocytes', 'lactate', 'hemoglobine', 'pas', 'pad', 'pam', 'fio2_corr']

linear_interpolate_inside = ['tidal_volume', 'pep','nad_dose_poids', 'dobu_dose_poids', 'fio2','o2_flow']

category = ['admin_o2', 'mode_vent', 'neuro_status']

interpolate_df = poids_corr.to_pandas()

In [ ]:
missing_matrix = interpolate_df.isnull()
missing_matrix.to_parquet(f'{DATA_FOLD}/{VERSION}/2.clean_data/{DATASET}/temporal/missing_matrix.parquet')

In [ ]:
def compute_urine_rate(df: pd.DataFrame,
                       time_col: str = 'delta_hour',
                       urine_col: str = 'urine_output',
                       output_col: str = 'urine_rate') -> pd.DataFrame:
    df = df.copy()
    df = df.sort_values(by=time_col).reset_index(drop=True)

    df[output_col] = np.nan

    last_index = None
    last_time = None

    for i in range(len(df)):
        val = df.loc[i, urine_col]
        if val != 0:
            if last_index is None:
                # Depuis le début jusqu'à la première valeur non nulle
                interval = df.loc[i, time_col]
                if interval == 0:
                    rate = val
                    df.loc[i, output_col] = rate
                else:
                    rate = val / interval
                    df.loc[:i, output_col] = rate
            else:
                # Depuis la dernière mesure non nulle
                delta_t = df.loc[i, time_col] - df.loc[last_index, time_col]
                if delta_t == 0:
                    rate = val
                    df.loc[i, output_col] = rate
                else:
                    rate = val / delta_t
                    df.loc[last_index+1:i+1, output_col] = rate
            last_index = i
            last_time = df.loc[i, time_col]

    # Optionnel : interpolation linéaire pour lisser encore plus
    df[output_col] = df[output_col].interpolate(method='linear')

    return df

In [ ]:
interpolate_df.columns

In [ ]:
encounter_list = static.filter(pl.col('utcInTime').dt.year() > 2022)['encounterId'].unique().cast(dtype=pl.Int32).to_list()

In [ ]:
impute = True

if impute : 
    for encounter in tqdm(interpolate_df.encounterId.unique().tolist()):
        mask = interpolate_df.encounterId==encounter
        interpolate_df.loc[mask, linear_interpolate_both] = interpolate_df.loc[mask, linear_interpolate_both].interpolate(method='linear', limit_direction='both')
        interpolate_df.loc[mask, linear_interpolate_inside] = interpolate_df.loc[mask, linear_interpolate_inside].interpolate(method='linear', limit_area='inside')
        interpolate_df.loc[mask, category] = interpolate_df.loc[mask, category].ffill().bfill()
        mask = interpolate_df.encounterId==encounter
        subset = interpolate_df.loc[mask, ['delta_hour', 'urine_output']].copy()
        if subset['urine_output'].notna().any():
            urine_rate_df = compute_urine_rate(subset)
            interpolate_df.loc[mask, 'urine_rate'] = urine_rate_df['urine_rate'].values

    interpolate_df['is_ventilated'] = interpolate_df['mode_vent'].isin(['Controlled', 'Assisted', 'Weaning'])
    interpolate_df = interpolate_df.drop(columns=['endotracheal_tube'])
    interpolate_df.to_parquet(f'{DATA_FOLD}/{VERSION}/2.clean_data/{DATASET}/temporal/treated_all.parquet')
    if_missing_then_default = {'nad_dose_poids' : 0, 
                           'dobu_dose_poids' : 0, 
                           'urine_rate' : 0, 
                           'o2_flow': 0,
                           'mode_vent': 'non_renseigné',
                           'neuro_status' : 'non_renseigné',
                           'eer' : 'sans_eer'
                        }   
    for col,def_value in if_missing_then_default.items() :
        interpolate_df.loc[interpolate_df[col].isnull(),col] = def_value
    interpolate_df.to_parquet(f'{DATA_FOLD}/{VERSION}/2.clean_data/{DATASET}/temporal/treated_all_with_placeholder_values.parquet')

else :
    interpolate_df = pd.read_parquet(f'{DATA_FOLD}/{VERSION}/2.clean_data/{DATASET}/temporal/treated_all_with_placeholder_values.parquet')

In [ ]:
interpolate_df.loc[interpolate_df['spo2'].isnull() | interpolate_df['spo2'].isnull(), interpolate_df.columns[:15]]

In [ ]:
for col in interpolate_df.columns :
    sum_null = interpolate_df[col].isnull().sum()
    print(f'{sum_null} pour colonne {col}')

In [ ]:
minimal_24h_df = pl.DataFrame(interpolate_df.copy())
discretize_df = pl.DataFrame(interpolate_df.copy())

In [ ]:
minimal_encounter_list = minimal_24h_df.filter(pl.col('delta_hour')>=16)['encounterId'].unique().to_list()

In [ ]:
minimal_24h_df.columns

In [ ]:
mean_sd = ['spo2', 'temp', 'pas', 'pad', 'pam',    'fr',
 'heart_rate', 'dobu_dose_poids', 'nad_dose_poids',  'glyc_cap', 'tp',
 'creat', 'bili_tot', 'num_plq', 'leucocytes', 'lactate', 'hemoglobine',  'is_prone',
 'fio2_corr', 'urine_rate', 'is_ventilated', 'poids_suivi' ]

sum = [ 'urine_output', 'iv_input','plq', 'pfc', 'cgr']

max = ['delta_hour', 'tracheo']

mode = ['mode_vent','ecmo_type', 'neuro_status' ]

drop = ['pplat','fio2']

minimal_grouped = (minimal_24h_df
 .filter(
     pl.col('encounterId').is_in(minimal_encounter_list) & pl.col('delta_hour').is_between(0,24)
     )
 .group_by('encounterId')
 .agg(
        pl.max(max), pl.mean(mean_sd, ).name.suffix("_mean"), pl.std(mean_sd, ).name.suffix("_std"), pl.sum(sum), pl.col(mode).mode().first()
    )
.with_columns(
    pl.when(pl.col('tracheo').is_not_null()).then(True).otherwise(False).alias('tracheo'),
    pl.when(pl.col('ecmo_type').is_not_null()).then('ecmo_type').otherwise(pl.lit('0')).alias('ecmo_type')
)
.sort('encounterId'))

biology = {
    'glyc_cap' : [1,2], 
    'tp': [80,100],
    'creat' : [50,110], 
    'bili_tot' : [5,15], 
    'num_plq' : [150,450], 
    'leucocytes' : [4, 11], 
    'lactate' : [1, 2], 
    'hemoglobine': [12,16]}

for bio, limit in biology.items() :
    
    minimal_grouped = minimal_grouped.with_columns(
        pl.col(bio).fill_null(np.random.uniform(limit[0], limit[1])).round(1)
    )
minimal_grouped = minimal_grouped.with_columns(
    pl.col('temp').fill_null(37)
)

In [ ]:
static_modified = static.with_columns(
    (pl.col('date_deces') - pl.col('utcInTime')).dt.total_days().alias('death_diff')
).with_columns(
    (pl.col('death_diff') < 30).alias('inf_j30')).fill_null(False)

In [ ]:
static_modified

In [ ]:
minimal_grouped = minimal_grouped.drop_nulls()
static_minimal = static_modified.select('encounterId', 'gender', 'age','admission_type', 'inf_j30', 'los').cast({'encounterId' : pl.Int32})
minimal_grouped_no_null = minimal_grouped.join(static_minimal, on = 'encounterId', how='inner').drop_nulls()

discretize_df = discretize_df.with_columns(
    pl.when(pl.col('pep') >= 16).then(pl.lit('very_high'))
        .when(pl.col('pep').is_between(12,16,closed='left')).then(pl.lit('high'))
        .when(pl.col('pep').is_between(8,12,closed='left')).then(pl.lit('moderate'))
        .when(pl.col('pep')<8).then(pl.lit('low'))
        .otherwise(pl.lit('No PEP')).alias('discr_pep'),
     pl.col('tidal_volume')/(pl.col('pplat')- pl.col('pep')).alias('compliance')
)

In [ ]:
minimal_grouped_no_null.write_parquet(f'{DATA_FOLD}/{VERSION}/2.clean_data/{DATASET}/static/minimal_grouped_24h.parquet')